# 機械学習 第３回レポート課題

## 0.概要
- 配布する訓練データ（X_train.npy、y_train.npy）を用いて2クラス分類問題を学習させ、配布するテストデータ（X_test.npy）に対するyのバイナリの予測値を一次元配列の形でcsvファイルとして提出せよ。授業で学んだようなモデルたちの比較や、ハイパーパラメーターを色々サーチしたりすることでベストなマイモデルを作成し、それによる予測値を提出。ファイル名は22vr0000_JeffHinton_Jeff.csvのように、学籍番号_名前_名前2.csvとすること。「名前2」の部分はスコアと共に公開されるので、どうしても本名が嫌な場合はあだ名を自分でつけること（まぁ、結果が悪くても恥ずかしがらなくていいので、本名で構いませんよ！）。利用可能な機械学習モデルは、授業で扱ったものに限るものとする。

- 訓練用のx、y（X_train.npy、y_train.npy）と、テスト用のxデータ（X_test.npy）は、以下のフォルダにある
    - https://drive.google.com/drive/folders/1DBoFa7Re_MRRrAjdvlOKDlwpYdfHQIyR?usp=share_link

### 注意事項
1. 再現性の確認のため、作業の記録・分析をColab Notebookの形にまとめてレポートとして提出すること。
2. 手抜き・工夫のない故に性能が低い結果は評価が低いが、いろいろがんばった結果スコアが悪い場合はきちんと評価する。また簡単に多量のモデルを比較できる便利ツールを使った結果は評価しません（便利で良いツールでも、それを使っても学びはないのと思うので、この授業では評価しません）

## 1.モデル構築用スクリプト

In [1]:
#必要ライブラリのインポート

import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier

In [2]:
#訓練用データセットの読み込み。
X_train = np.load('./report_3_dataset/X_train.npy')
y_train = np.load('./report_3_dataset/y_train.npy')

#DataFrameに変換
df_X = pd.DataFrame(X_train)
df_y = pd.DataFrame(y_train)

In [3]:
df_y

,0
0,0
1,0
2,0
3,0
4,0
...,...
500995,1
500996,1
500997,1
500998,1


In [4]:
df_X.describe()

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
count,501000.000000,501000.000000,501000.000000,501000.000000,501000.000000,501000.000000,501000.000000,501000.000000,501000.000000,501000.000000,...,501000.000000,501000.000000,501000.000000,501000.000000,501000.000000,501000.000000,501000.000000,501000.000000,501000.000000,501000.000000
mean,-0.296658,-0.000339,-0.000096,-0.353460,-0.000187,-0.167439,-0.620327,-0.001115,0.000116,-0.003484,...,-0.313463,0.001511,-0.000276,0.013857,-0.120519,-0.258847,-0.112201,-0.379788,-0.558423,1000.040912
std,0.800311,1.036417,1.001021,0.830636,0.999340,0.903298,0.609848,1.054332,1.000462,1.000990,...,0.779508,1.011269,0.999578,1.007975,0.841278,0.699219,0.670241,0.612964,0.710609,353.649359
min,-1.960549,-2.365194,-1.732160,-7.957071,-1.732132,-1.054221,-2.639252,-2.757771,-1.732348,-1.325801,...,-1.728273,-2.281851,-1.731758,-0.573682,-3.553998,-4.119920,-12.909924,-2.890791,-2.622245,499.999969
25%,-0.883383,-0.776614,-0.867655,-0.822573,-0.864117,-1.054221,-1.051701,-0.754030,-0.867531,-1.325801,...,-0.902770,-0.735839,-0.866116,-0.573682,-0.550642,-0.575910,-0.352207,-0.768097,-1.084409,750.000000
50%,-0.313111,0.000098,-0.000777,-0.282744,0.001136,-0.005984,-0.669953,-0.002491,0.000724,0.754261,...,-0.362352,0.002914,-0.001527,-0.573682,-0.228603,-0.372959,-0.330174,-0.529713,-0.659657,1000.000000
75%,0.260358,0.773945,0.867475,0.200908,0.865307,-0.005984,-0.247089,0.749907,0.867678,0.754261,...,0.203735,0.739490,0.864589,1.743123,0.296859,-0.175329,-0.273218,-0.097020,-0.134894,1250.000000
max,3.250548,2.365192,1.732370,3.499624,1.731978,4.482618,2.684753,2.758158,1.731450,0.754261,...,4.067270,2.282209,1.732740,1.743123,6.111819,7.371399,10.964278,4.101110,3.554938,1500.000000


In [5]:
#訓練用データと検証用データの分割

X_train, X_val, y_train, y_val = train_test_split(df_X, df_y, test_size=0.2, random_state=42)
print("各データセットのサイズ")
print(f'X_train: {X_train.shape}, y_train: {y_train.shape}')
print(f'X_test: {X_val.shape}, y_test: {y_val.shape}')


各データセットのサイズ
X_train: (400800, 28), y_train: (400800, 1)
X_test: (100200, 28), y_test: (100200, 1)


In [6]:
#とりあえず決定木での分類を試みる。

model = DecisionTreeClassifier()

search_param = {
    'max_depth': [None, 20, 40, 60, 80, 100],
    'min_samples_split': [2,5,10],
    'min_samples_leaf' : [1,2,4],
    'criterion': ['gini', 'entropy'],
}

grid_search = GridSearchCV(model, search_param, cv=3, scoring='accuracy')
grid_search.fit(X_train, y_train) #Fitに時間がかかる。。。

print(f"最適なパラメータ設定：{grid_search.best_params_}")
print(f"最適なパラメータ設定のスコア：{grid_search.best_score_}")